In [ ]:
# https://github.com/wlgh325/python_crawling/blob/master/1.%20%EC%B9%B4%EC%B9%B4%EC%98%A4%EB%A7%B5%20%EB%A6%AC%EB%B7%B0%20%ED%8C%8C%EC%9D%B4%EC%8D%AC%20%ED%81%AC%EB%A1%A4%EB%A7%81/kakaoMap_review_crawling.py
import os
from time import sleep

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

In [ ]:
##############################################################  ############
##################### variable related selenium ##########################
##########################################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


url = "https://map.kakao.com/"

#현재파일에 있는 크롬 드라이버를 가져와서 열기
options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
options.add_argument('headless') # 브라우저 안 띄우기
options.add_argument('lang=ko_KR') #  KR 언어
chromedriver_path = "chromedriver" # 크롬 드라이버 위치
#driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
driver = webdriver.Chrome(executable_path=r'C:/Users/anaco/Desktop/chromedriver-win64/chromedriver.exe')


In [ ]:
# 검색 list 만들어줌 li

df=pd.read_csv('exper.csv')
df['name']
#len(df['name'])
li=[]

for i in range(len(df['name'])):
    i=str(df['name'][i])+' 제주'
    li.append(i)
print(li)



In [ ]:
'''   내용 확인 코드------
for i in df['name']:
    na=str(i)
    place_infos=[na]
    '''

for i, place in enumerate(li):
    # delay
    if i % 4 == 0 and i != 0:
        sleep(5)
    print("#####", i, place)
            

In [ ]:
def main():
    global driver, load_wb, review_num
    
    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    #place_infos = ['강남 맛집']
    
    for i, place in enumerate(li):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(1)
        print("#####", i)
        search(place)
      
    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(0.5)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)########################### re부터 return까지 수정
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(0.5)

        # 2~ 5페이지 읽기 ~3
        for i in range(2, 5):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element(By.XPATH,xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()
        


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        
        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element(By.XPATH, detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)
        #driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a/span[1]').send_keys(Keys.ENTER)
        

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements(By.CLASS_NAME,'link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                #driver.find_element(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a/span[1]').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element(By.LINK_TEXT,'다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")
            
        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements(By.CLASS_NAME,'link_page'))
                for i in range(page_num-1):
                    driver.find_element(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element(By.LINK_TEXT,'다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


# 실질적으로 리뷰 모이는 부분
def extract_review(place_name):
    global driver

    ret = True
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
   

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            #comment = review.select('.list_evaluation > li> comment_info > txt_comment > span') # 리뷰############수정
            #rating = review.select('.list_evaluation > li > unit_info > txt_desc') # 별점############수정
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret   

if __name__ == "__main__":
    main()
    
    